In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
from glob import glob
import re
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
#from tqdm import tqdm
from scipy import stats
from sklearn.metrics import pairwise_distances

import settings as conf
# from src.data import PhenoInfo, PhenoResults, get_all_tissues, get_genes
from results.multixcan import MXPhenoInfo, MXPhenoResults
from utils import is_number, chunker

In [4]:
# genes_associations_dir = os.path.join(constants.PREPROCESSED_BASED_DIR, 'gene_associations')
# smultixcan_gene_association_dirs = os.path.join(genes_associations_dir, 'mashr')

# Load metadata

In [5]:
with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load MultiXcan associations

In [6]:
genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan-mashr-pvalues.pkl.xz')
display(genes_associations_filename)

genes_associations = pd.read_pickle(genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan-mashr-pvalues.pkl.xz'

In [7]:
display(genes_associations.shape)
display(genes_associations.head())

(22515, 4091)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.865429,0.918314,0.810683,0.374671,0.189032,0.140981,0.467741,0.129427,0.193680,0.285479,...,0.416210,0.782554,0.609467,0.980281,0.666504,0.409761,0.713310,0.168319,0.460244,0.765506
ENSG00000000457,0.174192,0.064765,0.889194,0.896938,0.448596,0.269602,0.540261,0.068405,0.041813,0.313427,...,0.149360,0.512603,0.010907,0.228982,0.607081,0.812484,0.678749,0.918971,0.311187,0.344574
ENSG00000000460,0.879969,0.240715,0.238228,0.567555,0.921320,0.825036,0.782230,0.644525,0.392273,0.840014,...,0.503520,0.764147,0.587969,0.301460,0.629621,0.486664,0.736509,0.933600,0.000477,0.321223
ENSG00000000938,0.192670,0.400054,0.114353,0.470700,0.889202,0.000011,0.899764,0.212352,0.829671,0.372348,...,0.899212,0.961678,0.059247,0.588855,0.898525,0.135045,0.954998,0.088220,0.176497,0.304281
ENSG00000000971,0.180632,0.793060,0.490585,0.088752,0.744531,0.949639,0.253817,0.377408,0.971655,0.070266,...,0.390618,0.093824,0.020391,0.109883,0.870551,0.995450,0.002660,0.421588,0.656851,0.868416


# Load PheWAS catalog

In [8]:
phewas_catalog = pd.read_csv(os.path.join(conf.DATA_DIR, 'phewas-catalog.csv.gz'), dtype={'phewas code': str})

In [9]:
phewas_catalog.shape

(215107, 9)

In [10]:
phewas_catalog[phewas_catalog['phewas code'].isna()].head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations


In [11]:
phewas_catalog[phewas_catalog['gene_name'].isna()].head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations
41,4 111710169,rs2200733,Atrial fibrillation,1950,1.527000e-10,1.5170,NaN,427.21,"Atrial fibrillation, Atrial fibrillation/atria..."
49,4 111710169,rs2200733,Atrial fibrillation & flutter,2041,1.019000e-09,1.4810,NaN,427.2,"Atrial fibrillation, Atrial fibrillation/atria..."
98,4,rs4698036,Gout,769,7.803000e-08,0.6839,NaN,274.1,Serum uric acid
108,4,rs4698036,Gout and other crystal arthropathies,904,1.990000e-07,0.7132,NaN,274,Serum uric acid
115,8 128485038,rs1447295,Prostate cancer,848,2.758000e-07,1.6060,NaN,185,Prostate cancer


In [12]:
phewas_catalog[phewas_catalog['gene_name'].isna()].shape

(52140, 9)

In [13]:
phewas_catalog = phewas_catalog.dropna(subset=['gene_name', 'phewas code'])

In [14]:
phewas_catalog.shape

(162967, 9)

In [15]:
phewas_catalog['gene_name'].unique().shape

(1775,)

In [16]:
phewas_catalog['phewas code'].unique().shape

(1358,)

In [17]:
phewas_catalog = phewas_catalog.assign(gene_id=phewas_catalog['gene_name'].apply(lambda x: genes_mapping_1[x] if x in genes_mapping_1 else None))

In [18]:
phewas_catalog = phewas_catalog.dropna(subset=['gene_name', 'gene_id', 'phewas code'])

In [19]:
phewas_catalog.shape

(147970, 10)

In [20]:
phewas_catalog.head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations,gene_id
0,19 45395619,rs2075650,Alzheimer's disease,737,5.237000e-28,2.410,TOMM40,290.11,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204
1,19 45395619,rs2075650,Dementias,1170,2.409000e-26,2.114,TOMM40,290.1,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204
2,6 396321,rs12203592,Actinic keratosis,2505,4.141000e-26,1.691,IRF4,702.1,"Eye color, Hair color, Freckling, Progressive ...",ENSG00000137265
3,6 26093141,rs1800562,Iron metabolism disorder,40,3.409000e-25,12.270,HFE,275.1,"Mean corpuscular hemoglobin, Glycated hemoglob...",ENSG00000010704
4,19 45395619,rs2075650,Delirium dementia and amnestic disorders,1566,8.027000e-24,1.841,TOMM40,290,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204


In [21]:
phewas_catalog.sort_values('phewas phenotype').head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations,gene_id
35306,10,rs7923609,ASCVD,166,0.008094,1.3610,JMJD1C,414.2,Alkaline phosphatase,ENSG00000171988
154790,22,rs1012068,ASCVD,166,0.035970,1.2920,DEPDC5,414.2,Chronic Hepatitis C infection,ENSG00000100150
72358,5 158814533,rs10045431,ASCVD,166,0.016740,0.7242,IL12B,414.2,Crohn's disease,ENSG00000113302
130720,14 87896435,rs17124581,ASCVD,166,0.030370,1.6090,SPATA7,414.2,Cognitive performance,ENSG00000042317
184453,6 31912648,rs429608,ASCVD,166,0.042840,1.3440,SKIV2L,414.2,Age-related macular degeneration,ENSG00000204351


# Genes in common

In [22]:
shared_gene_ids = \
    set(phewas_catalog['gene_id'].values)\
    .intersection(genes_associations.index)

In [23]:
len(shared_gene_ids)

1593

# HPO to MIM

In [24]:
hpo_to_mim = pd.read_csv(os.path.join(conf.DATA_DIR, 'hpo-to-omim-and-phecode.csv.gz'), dtype={'phecode': str})

In [25]:
hpo_to_mim.shape

(84031, 10)

In [26]:
hpo_to_mim.head()

,term_id,name,match_available,phecode,phecode string,match_type,class,dID,disease_name,modifier
0,28,Cryptorchidism,1,751.12,Congenital anomalies of male genital organs,General,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
1,49,Shawl scrotum,1,751.12,Congenital anomalies of male genital organs,General,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
2,175,Cleft palate,1,749.1,Cleft palate,Exact,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
3,202,Oral cleft,1,749.1,Cleft palate,Broader,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
4,204,Cleft upper lip,1,749.1,Cleft palate,Broader,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O


# Load silver standard to map from UKB to MIM

In [27]:
omim_silver_standard = pd.read_csv(os.path.join(conf.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [28]:
ukb_to_mim_map = omim_silver_standard[['trait', 'pheno_mim']].dropna()

In [29]:
ukb_to_mim_map.shape

(7822, 2)

In [30]:
ukb_to_mim_map.head()

,trait,pheno_mim
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145


# Read gwas2gene results

In [31]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [32]:
readRDS = robjects.r['readRDS']

In [33]:
f_files = glob(os.path.join(conf.OMIM_SILVER_STANDARD_GWAS_TO_GENE_DIR, '*.rds'))
display(len(f_files))

if len(f_files) != len(omim_silver_standard['trait'].unique()):
    print(f'WARNING: some files are not there. {len(omim_silver_standard.trait.unique())} expected, {len(f_files)} found.')

99

In [34]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code = f_base.split('.')[0]
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = f_gene_list

In [35]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

gwas2gene_all_genes = shared_gene_ids.intersection(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

20837

10185

958

In [36]:
pd.Series(list(gwas2gene_all_genes)).head()

0    ENSG00000135074
1    ENSG00000169862
2    ENSG00000123374
3    ENSG00000140718
4    ENSG00000100147
dtype: object

# Merge

In [37]:
# mim to phecode
_tmp = pd.merge(ukb_to_mim_map, hpo_to_mim[['phecode', 'dID']], left_on='pheno_mim', right_on='dID', how='inner').drop(columns=['dID'])
display(_tmp.shape)

(154338, 3)

In [38]:
# phecode to phewas catalog
_tmp = pd.merge(_tmp, phewas_catalog[['phewas code', 'gene_name', 'gene_id']], left_on='phecode', right_on='phewas code', how='inner').drop(columns=['phewas code'])
display(_tmp.shape)

(10256175, 5)

In [39]:
_tmp.head()

,trait,pheno_mim,phecode,gene_name,gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,GLG1,ENSG00000090863
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,STIM2,ENSG00000109689
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,MLPH,ENSG00000115648


In [40]:
# just wondering if this is the right way... but I don't think so, I want just a ukb-trait to gene map and see how many are significant
#_tmp.drop_duplicates(subset=['trait', 'phecode', 'gene_id']).shape

In [41]:
_tmp = _tmp.drop_duplicates(subset=['trait', 'gene_id'])

In [42]:
_tmp.shape

(153579, 5)

In [43]:
from entity import Trait

In [44]:
_tmp = _tmp.assign(ukb_code=[Trait(full_code=t).code for t in _tmp['trait'].values])

In [45]:
_tmp.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344,M41
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,GLG1,ENSG00000090863,M41
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,STIM2,ENSG00000109689,M41
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,MLPH,ENSG00000115648,M41
5,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,JMJD1C,ENSG00000171988,M41


In [46]:
phewas_ukb_results = _tmp

### Keep genes per trait that we can find in GWAS

In [47]:
trait_genes_to_keep = []

# idx = 0
for trait, data in phewas_ukb_results.groupby(['trait']):
    ukb_trait_code = data.ukb_code.unique()[0]
    
    if ukb_trait_code in gwas2genes_results.keys():
        ukb_traits_genes = gwas2genes_results[ukb_trait_code]
    else:
        print(f'WARNING: for ukb trait "{trait}", no genes were found')
        continue
    
    ukb_traits_genes = set(ukb_traits_genes)
    ukb_traits_genes = ukb_traits_genes.intersection(gwas2gene_all_genes)
    
    idx_to_keep = data[data['gene_id'].isin(ukb_traits_genes)].index.tolist()
    trait_genes_to_keep.extend(idx_to_keep)

In [48]:
display(len(trait_genes_to_keep))

2205

In [49]:
display(trait_genes_to_keep[:5])

[319791, 319794, 3342736, 3342786, 3444215]

In [50]:
phewas_ukb_results = phewas_ukb_results.loc[trait_genes_to_keep]

In [51]:
phewas_ukb_results.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code
319791,1200-Sleeplessness_insomnia,168605,378,IKZF4,ENSG00000123411,1200
319794,1200-Sleeplessness_insomnia,168605,378,PRIM1,ENSG00000198056,1200
3342736,1200-Sleeplessness_insomnia,121300,563,ABCG8,ENSG00000143921,1200
3342786,1200-Sleeplessness_insomnia,121300,563,THADA,ENSG00000115970,1200
3444215,1200-Sleeplessness_insomnia,600072,331,MEIS1,ENSG00000143995,1200


In [52]:
phewas_ukb_results = phewas_ukb_results.assign(multixcan_pval=np.nan)

for i, d in phewas_ukb_results.iterrows():
    gene_id = d['gene_id']
    if gene_id not in genes_associations.index:
        continue
    
    ukb_fullcode = d['trait']
    gene_trait_pval = genes_associations.loc[gene_id, ukb_fullcode]
    phewas_ukb_results.loc[i, 'multixcan_pval'] = gene_trait_pval

In [53]:
phewas_ukb_results.shape

(2205, 7)

In [54]:
phewas_ukb_results = phewas_ukb_results.dropna(subset=['multixcan_pval'])

In [55]:
phewas_ukb_results.shape

(2202, 7)

In [56]:
phewas_ukb_results.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code,multixcan_pval
319791,1200-Sleeplessness_insomnia,168605,378,IKZF4,ENSG00000123411,1200,0.000826
319794,1200-Sleeplessness_insomnia,168605,378,PRIM1,ENSG00000198056,1200,0.231724
3342736,1200-Sleeplessness_insomnia,121300,563,ABCG8,ENSG00000143921,1200,0.136159
3342786,1200-Sleeplessness_insomnia,121300,563,THADA,ENSG00000115970,1200,0.100410
3444215,1200-Sleeplessness_insomnia,600072,331,MEIS1,ENSG00000143995,1200,0.000049


In [57]:
phewas_ukb_results.multixcan_pval.describe().apply(str)

count                    2202.0
mean        0.20234246892891014
std         0.28413966610318014
min                      1e-320
25%      1.4824595405000882e-06
50%        0.027428990301668253
75%          0.3435073855718861
max          0.9982749103428394
Name: multixcan_pval, dtype: object

In [58]:
phewas_ukb_results[phewas_ukb_results['trait'] == '1200-Sleeplessness_insomnia'].shape

(48, 7)

### Count

In [59]:
total_phewas_catalog = phewas_ukb_results.shape[0]
display(total_phewas_catalog)

2202

In [60]:
# 0.05
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < 0.05].shape[0]
display(hits)

1179

In [61]:
(hits / total_phewas_catalog) * 100.0

53.54223433242507

In [62]:
# 0.01
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < 0.01].shape[0]
display(hits)

1005

In [63]:
(hits / total_phewas_catalog) * 100.0

45.64032697547684

In [64]:
SIGNIFICANT_PVALUE = 0.05 / total_phewas_catalog
display(SIGNIFICANT_PVALUE)

2.270663033605813e-05

In [65]:
# pvalue threshold
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < SIGNIFICANT_PVALUE].shape[0]
display(hits)

664

In [66]:
(hits / total_phewas_catalog) * 100.0

30.154405086285195